# Diffie Hellman

- Establish a shared secret between two parties over an insecure channel.
- Provides confidentiality foundation for symmetric encryption; does not provide authentication/confidentiality by itself.

The key is to use public encryption to solve the key distribution 

- Distribution of public keys via certificates with a Third Trusted Party
    - Can be donde using:
        - Public announcement 
        - Public available directory
        - Public Key Authority
        - Centralized way? no bottleneck distributed
- Use of public jet encryption to distribute secret keys



### Primitive root (notes)

- Definition  
    A number $g$ is a primitive root modulo $n$ if the powers of $g$ generate all units modulo $n$ (the multiplicative group $(\mathbb{Z}/n\mathbb{Z})^*$). Equivalently, the multiplicative order of $g$ modulo $n$ equals $\phi(n)$ (Euler's totient).

- Important special case (primes)  
    For a prime $p$ the group $(\mathbb{Z}/p\mathbb{Z})^*$ is cyclic of order $p-1$, so a primitive root modulo $p$ is an element of order $p-1$. Primitive roots always exist for prime moduli.

- Practical test for primes  
    Let $p$ be prime and let the distinct prime factors of $p-1$ be $q_1, q_2, \dots, q_k$. A candidate $g$ ($2 \le g < p$) is a primitive root mod $p$ iff for every $i$:

    $g^{(p-1)/q_i} \not\equiv 1 \pmod p.$

    If any test equals $1$, $g$ is not a generator.

- How to find one (algorithm sketch)
    1. Factor $p-1$ to get its prime factors $\{q_i\}$.  
    2. For $g = 2, 3, 4, \dots$ check $g^{(p-1)/q_i} \pmod p$ for all $q_i$.  
    3. The first $g$ that passes all checks is a primitive root.

- Small example (p = 7)  
    $p-1 = 6$, prime factors $\{2, 3\}$. Test $g = 3$:  

    $3^{6/2} = 3^3 \equiv 6 \pmod 7$
    $3^{6/3} = 3^2 \equiv 2 \pmod 7$  

    So $3$ is a primitive root mod $7$. Powers: $3^1\dots 3^6 \equiv 3,2,6,4,5,1$ (all nonzero residues).

- Relevance to Diffie–Hellman  
    In Diffie–Hellman you pick a prime $p$ and a generator $g$ (a primitive root modulo $p$ or at least an element of large order) so that repeated powers of $g$ produce the key space used for the exchange.

In [29]:
def gcd(a, b):
    """Compute the greatest common divisor of a and b."""
    print(f"Computing GCD of {a} and {b}")
    while b:
        print(f"gcd({a}, {b})")
        a, b = b, a % b
    
    print(f"gcd({a}, {b})")
    return a

In [28]:
def verify_primitive_root(root, p):
    """Verify if root is a primitive root modulo p."""
    for k in range(1, p+4):
        print(f"Computing {root}^{k} is {pow(root, k)} mod {p} = {pow(root, k)% p}")

print("Verifying if 2 is a primitive root modulo 11:")
verify_primitive_root(2, 11)

Verifying if 2 is a primitive root modulo 11:
Computing 2^1 is 2 mod 11 = 2
Computing 2^2 is 4 mod 11 = 4
Computing 2^3 is 8 mod 11 = 8
Computing 2^4 is 16 mod 11 = 5
Computing 2^5 is 32 mod 11 = 10
Computing 2^6 is 64 mod 11 = 9
Computing 2^7 is 128 mod 11 = 7
Computing 2^8 is 256 mod 11 = 3
Computing 2^9 is 512 mod 11 = 6
Computing 2^10 is 1024 mod 11 = 1
Computing 2^11 is 2048 mod 11 = 2
Computing 2^12 is 4096 mod 11 = 4
Computing 2^13 is 8192 mod 11 = 8
Computing 2^14 is 16384 mod 11 = 5


# Finding other primitive roots

If $a$ is a primitive root modulo a prime $p$, then for any integer $r$ the element $a^r$ is a primitive root modulo $p$ if and only if $\gcd(r, p-1) = 1$.

Why: the order of $a$ modulo $p$ is $p-1$. The order of $a^r$ is $\frac{p-1}{\gcd(r,p-1)}$. So $a^r$ has order $p-1$ (i.e. is a primitive root) exactly when $\gcd(r,p-1)=1$.

Consequence: all primitive roots modulo $p$ are obtained as $a^r$ where $r$ ranges over the integers coprime to $p-1$. The number of primitive roots modulo $p$ is $\phi(p-1)$ (Euler's totient of $p-1$).

Example (p = 59, a = 2):

- Here $p-1 = 58 = 2 dot 29$. The integers coprime to 58 are those not divisible by 2 or 29. There are $\phi(58)=28$ such values, so there are 28 primitive roots modulo 59.

- To build other primitive roots from $2$, compute $2^r \mod 59$ for each $r$ with $1 e r < 58$ and $\gcd(r,58)=1$. A few examples:
    - $r=3$: $2^3 = 8$ so $8$ is a primitive root mod 59.
    - $r=5$: $2^5 = 32$ so $32$ is a primitive root mod 59.
    - $r=7$: $2^7 = 128 quiv 10 mod{59}$ so $10$ is a primitive root mod 59.

Practical algorithm to list primitive roots given one primitive root $a$:
1. Factor $p-1$ and compute the list $R = r : 1 e r < p-1,\ cd(r,p-1)=1$.
2. For each $r$ in $R$ compute $a^r \mod p$; the results (distinct) are exactly the primitive roots modulo $p$.

Note: when implementing this, avoid duplicates (different $r$ can give the same residue); using a set to collect results handles that automatically.



In [30]:
def find_primitive_root(a, p):
    """Find a primitive root modulo p."""
    primitive_roots = []

    # Find all coprimes to p -1
    print(f"Finding coprimes to {p-1}")
    coprimes = [i for i in range(1, p-1) if gcd(i, p-1) == 1]

    print(f"Coprimes to {p-1}: {coprimes}")

    print(f"Calculating primitive roots using base {a} modulo {p}")
    print(f"This works because it is known that if a is a primitive root modulo p, then a^k mod p is also a primitive root if k is coprime to p-1.")
    for k in coprimes:
        primitive_roots.append(a**k % p)  


    print(f"Primitive roots modulo {p}: {primitive_roots}")
    return primitive_roots

find_primitive_root(2,11)

Finding coprimes to 10
Computing GCD of 1 and 10
gcd(1, 10)
gcd(10, 1)
gcd(1, 0)
Computing GCD of 2 and 10
gcd(2, 10)
gcd(10, 2)
gcd(2, 0)
Computing GCD of 3 and 10
gcd(3, 10)
gcd(10, 3)
gcd(3, 1)
gcd(1, 0)
Computing GCD of 4 and 10
gcd(4, 10)
gcd(10, 4)
gcd(4, 2)
gcd(2, 0)
Computing GCD of 5 and 10
gcd(5, 10)
gcd(10, 5)
gcd(5, 0)
Computing GCD of 6 and 10
gcd(6, 10)
gcd(10, 6)
gcd(6, 4)
gcd(4, 2)
gcd(2, 0)
Computing GCD of 7 and 10
gcd(7, 10)
gcd(10, 7)
gcd(7, 3)
gcd(3, 1)
gcd(1, 0)
Computing GCD of 8 and 10
gcd(8, 10)
gcd(10, 8)
gcd(8, 2)
gcd(2, 0)
Computing GCD of 9 and 10
gcd(9, 10)
gcd(10, 9)
gcd(9, 1)
gcd(1, 0)
Coprimes to 10: [1, 3, 7, 9]
Calculating primitive roots using base 2 modulo 11
This works because it is known that if a is a primitive root modulo p, then a^k mod p is also a primitive root if k is coprime to p-1.
Primitive roots modulo 11: [2, 8, 7, 6]


[2, 8, 7, 6]

## Diffi Hellman Key Exchange Algorithm

The goal is to enable two uses to securely exchange a key 

Each user generate their key 
- chooses a secret key numbre 
- compute their public key ya = a^xa mod p
- compute their public key yb = a^xb mod p
- To get the session key for users 
- KAb = a^xaxb mod p
= ya^xb mod  p  b compute
= yb^xa mod  p  b compute


In [31]:
def diffie_hellman_given_private_keys(p, g, a_private, b_private):
    """Perform Diffie-Hellman key exchange."""
    print(f"Using prime p = {p} and base g = {g}")
    print(f"Given private keys: a = {a_private}, b = {b_private}")
    print(f"Calculating public keys:")
    a_public = pow(g, a_private, p)
    print(f"Alice's public key: g ^ a mod p = {g}^{a_private} mod {p} = {a_public}")
    b_public = pow(g, b_private, p)
    print(f"Bob's public key: g ^ b mod p = {g}^{b_private} mod {p} = {b_public}")

    print(f"Calculating shared secret keys:")
    a_shared_secret = pow(b_public, a_private, p)
    print(f"Alice's shared secret: B^{a_private} mod {p} = {a_shared_secret}")
    b_shared_secret = pow(a_public, b_private, p)
    print(f"Bob's shared secret: A^{b_private} mod {p} = {b_shared_secret}")

    assert a_shared_secret == b_shared_secret, "Shared secrets do not match!"
    print(f"Kab secret established: {a_shared_secret}")
    return a_shared_secret

In [32]:
def logarithm_discrete(g, h, p):
    """Compute the discrete logarithm of h to the base g modulo p."""
    print(f"Computing discrete logarithm of {h} to the base {g} modulo {p}")
    for x in range(p):
        computed = pow(g, x, p)
        print(f"g^{x} mod {p} = {computed}")
        if computed == h:
            print(f"Found discrete logarithm: x = {x}")
            return x
    raise ValueError("Logarithm not found")

In [33]:
def diffie_hellman_given_public_keys(p, g, a_public, b_public):
    """Perform Diffie-Hellman key exchange."""
    print(f"Using prime p = {p} and base g = {g}")
    print(f"Given public keys: A = {a_public}, B = {b_public}")
    print(f"Calculating shared secret keys:")
    a_private = logarithm_discrete(g, a_public, p)
    print(f"Alice's private key found: {a_private}")
    b_private = logarithm_discrete(g, b_public, p)
    print(f"Bob's private key found: {b_private}")
    a_shared_secret = pow(b_public, a_private, p)
    print(f"Alice's shared secret: B ^ a mod p = {b_public}^{a_private} mod {p} = {a_shared_secret}")
    b_shared_secret = pow(a_public, b_private, p)
    print(f"Bob's shared secret: A ^ b mod p = {a_public}^{b_private} mod {p} = {b_shared_secret}")
    assert a_shared_secret == b_shared_secret, "Shared secrets do not match!"
    print(f"Kab secret established: {a_shared_secret}")
    return a_shared_secret



diffie_hellman_given_private_keys(353, 3, 97, 233)

diffie_hellman_given_public_keys(11, 2, 5, 9)


Using prime p = 353 and base g = 3
Given private keys: a = 97, b = 233
Calculating public keys:
Alice's public key: g ^ a mod p = 3^97 mod 353 = 40
Bob's public key: g ^ b mod p = 3^233 mod 353 = 248
Calculating shared secret keys:
Alice's shared secret: B^97 mod 353 = 160
Bob's shared secret: A^233 mod 353 = 160
Kab secret established: 160
Using prime p = 11 and base g = 2
Given public keys: A = 5, B = 9
Calculating shared secret keys:
Computing discrete logarithm of 5 to the base 2 modulo 11
g^0 mod 11 = 1
g^1 mod 11 = 2
g^2 mod 11 = 4
g^3 mod 11 = 8
g^4 mod 11 = 5
Found discrete logarithm: x = 4
Alice's private key found: 4
Computing discrete logarithm of 9 to the base 2 modulo 11
g^0 mod 11 = 1
g^1 mod 11 = 2
g^2 mod 11 = 4
g^3 mod 11 = 8
g^4 mod 11 = 5
g^5 mod 11 = 10
g^6 mod 11 = 9
Found discrete logarithm: x = 6
Bob's private key found: 6
Alice's shared secret: B ^ a mod p = 9^4 mod 11 = 5
Bob's shared secret: A ^ b mod p = 5^6 mod 11 = 5
Kab secret established: 5


5